In [1]:
import pandas as pd

nsmc_fname = 'ratings.txt'
nsmc_total_data = pd.read_csv(nsmc_fname, sep='\t')
text_data_list = nsmc_total_data['document'].dropna().values

print(text_data_list[:4])

['어릴때보고 지금다시봐도 재밌어요ㅋㅋ'
 '디자인을 배우는 학생으로, 외국디자이너와 그들이 일군 전통을 통해 발전해가는 문화산업이 부러웠는데. 사실 우리나라에서도 그 어려운시절에 끝까지 열정을 지킨 노라노 같은 전통이있어 저와 같은 사람들이 꿈을 꾸고 이뤄나갈 수 있다는 것에 감사합니다.'
 '폴리스스토리 시리즈는 1부터 뉴까지 버릴께 하나도 없음.. 최고.'
 '와.. 연기가 진짜 개쩔구나.. 지루할거라고 생각했는데 몰입해서 봤다.. 그래 이런게 진짜 영화지']


In [2]:
from gensim.models.doc2vec import Doc2Vec, TaggedDocument
from konlpy.tag import Mecab; mecab=Mecab()
from tqdm import tqdm

tagged_data = [TaggedDocument(words=mecab.morphs(xx), tags=str(ii)) for ii, xx in tqdm(enumerate(text_data_list))]

print('\n', tagged_data[0])

199992it [00:14, 13566.52it/s]


 TaggedDocument(['어릴', '때', '보', '고', '지금', '다시', '봐도', '재밌', '어요', 'ㅋㅋ'], 0)


In [3]:
import time

max_epochs = 5
d2v_model_fname = 'nsmc_d2v_model.model'

d2v_model = Doc2Vec(vector_size=256, alpha=0.025, min_alpha=0.00025, min_count=10, dm=1)
d2v_model.build_vocab(tagged_data)

# d2v_model.train(tagged_data, epochs=5, total_examples=d2v_model.corpus_count)
s_time = time.time()
for epoch in range(max_epochs):
    print('epoch : %d' % (epoch+1,))
    d2v_model.train(tagged_data, total_examples=d2v_model.corpus_count, epochs=d2v_model.iter)

    d2v_model.alpha -= 0.0002
    d2v_model.min_alpha = d2v_model.alpha

d2v_model.save(d2v_model_fname)
print('%dm %.2fs' % divmod(time.time()-s_time, 60))

epoch : 1


/root/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:13: DeprecationWarning: Call to deprecated `iter` (Attribute will be removed in 4.0.0, use self.epochs instead).
  del sys.path[0]


epoch : 2
epoch : 3
epoch : 4
epoch : 5
3m 53.59s


In [4]:
d2v_model = Doc2Vec.load(d2v_model_fname)

print(d2v_model.infer_vector(mecab.morphs(sample_text_data_list[0])).shape)
print(d2v_model.docvecs.most_similar(0))
print(d2v_model.docvecs['0'].shape)

(256,)
[('5', 0.1890677511692047), ('2', 0.15747705101966858), ('4', 0.12792299687862396), ('1', 0.11842948198318481), ('6', 0.10007698833942413), ('7', 0.09417770802974701), ('3', 0.059770748019218445), ('8', 0.02612971141934395), ('9', -0.03948981314897537)]
(256,)


In [5]:
print(d2v_model.wv.most_similar('평점', topn=4))
 
print(d2v_model.wv.most_similar('평점', topn=4))
print(d2v_model.wv.similarity('평점', '별점'))

print(d2v_model.wv['평점'].shape)

[('별점', 0.5890220403671265), ('평정', 0.5434197783470154), ('평', 0.5378557443618774), ('점수', 0.46702587604522705)]
[('별점', 0.5890220403671265), ('평정', 0.5434197783470154), ('평', 0.5378557443618774), ('점수', 0.46702587604522705)]
0.58902204
(256,)
